## 3. TCP 网络通信

>   网络调试助手

### 3.1 IP 地址

>   ssh 用户名@服务器IP（域名）

作用: IP就是标识 计算机网络中一个主机设备

分类: IPv4 			4个字节(32bit)描述   40 多亿

​		x.x.x.x  			x    0-255

​	和 IPv6      

命令操作

查看网卡信息(IP 地址)  

​	Linux Mac   `ifconfig`

​	win             `ipconfig /all`

测试与远程主机的网络连通性

​	ping 主机IP/主机域名

>   域名在互联网中最终都会被转化为 IP   转化过程称为DNS解析
>
>   DNS解析由互联网中的 DNS 服务器完成



### 3.2 端口

>   需要网络通信的程序一定需要一个端口 0-65535

​	作用: 标识主机设备中一个应用程序(服务)

​	分类: 

​		知名端口 0-1023     `  HTTP80 HTTPS 443  SSH22`

[http://www.baidu.com:80](http://www.baidu.com/)   [https://www.taobao.com:443](https://www.baidu.com/)

​		动态端口 1024-65535

​	命令-扩展:

​		`sudo lsof -i :22` 查看 22 号端口被谁占用

```shell
python@ubuntu:~/workspace/28多任务$ sudo lsof -i :22
[sudo] python 的密码： 
COMMAND  PID USER   FD   TYPE   DEVICE SIZE/OFF NODE NAME
sshd    1139 root    3u  IPv4 15178455      0t0  TCP *:ssh (LISTEN)
sshd    1139 root    4u  IPv6 15178457      0t0  TCP *:ssh (LISTEN)
```

### 3.3 socket

套接字、网络插座

>   网络通信    是对复杂的网络通信功能的封装 易用

### 3.4 TCP套接字特点

>   tcp socket  就是一种可以通信的套接字

全称  传输控制协议

-   面向连接: 建立连接、通信、关闭连接<类似打电话>
-   可靠  

    -   应答机制    收到数据回复一个应答ACK

    -   超时重传    发送方启动定时器在固定时间内没有收到 ACK  重传

    -   错误校验    发送方会数据和标记一起发送 

    -   拥塞控制    根据接收方的速度动态调整发送的速度
-   基于字节流  数据的基本单位是字节
    -   A 给 B 先发了"A",  又发了 "B", 又发了'C'
    -   B 并不能区分 A 发了几次 只能接收到 "ABC"字节流
-   传输层协议  

    -   TCP/IP 四层参考模型中  在上层有一个应用层

### 3.5 编解码-重点

```
# str --编码　字符串.encode(encoding='utf-8')---> bytes
#   <----解码 字节数据.decode(encoding='utf-8')--
```

## 4. TCP通信过程-重点

>   client 客户端     服务器 server       b/s  c/s

### 4.1 客户端通信过程

```python
import socket


def main():
    # 1 买个电话用以通信　　　创建套接字对象(IPv4   基于字节流STREAM)
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2 拨号               建立和服务器程序的连接　IP+端口组合
    # 参数是IP和端口的元组
    server_address = ('172.16.51.151', 8080)
    tcp_socket.connect(server_address)

    # 3 发送数据  .send(需要发送的字节数据)
    tcp_socket.send('helloworld'.encode())

    # 4 接收数据 参数就是本次接收数据的最大字节数 返回值就是接收到的字节数据
    recv_data = tcp_socket.recv(1024)
    # print(ret)
    print("收到来自服务器的数据", recv_data.decode())   #　收到的数据是字节　需要解码

    # 5 通信完成　挂机
    tcp_socket.close()
if __name__ == '__main__':
    main()
```



```python
import socket

套接字对象 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

套接字对象.connect(('服务器IP', 服务器端口号))

套接字对象.send(需要发送的字节数据)
字节数据 = TCP 对象.recv(本次接收数据的最大字节数)

套接字对象.close()
```



### 4.2 recv返回值说明

作用  TCP接收数据

说明  字节数据 = TCP 对象.recv(本次接收数据的最大字节数)

返回值   如果对方断开连接  字节数据将会是 b''

​			 正常                      字节数据将会是对方发送给本机的内容

```python
import socket


def main():
    # 1 买个电话用以通信、创建套接字对象(IPv4、基于字节流STREAM)
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2 拨号               建立和服务器程序的连接　IP+端口组合
    # 参数是IP和端口的<元组>  IP是字符串　端口是整数
    tcp_socket.connect(('172.16.51.151', 8080))

    # 3 发送数据  .send(需要发送的字节数据)
    tcp_socket.send('helloworld'.encode())

    # 4 接收数据 参数就是本次接收数据的最大字节数 正常情况　返回值就是接收到的字节数据;
    # 如果对方断开了连接　　接收到数据将会是空的 '' （!= ' '）　收到长度为０的空数据
    recv_data = tcp_socket.recv(1024)
    if recv_data:
        print("收到来自服务器的数据", recv_data.decode())  # 收到的数据是字节　需要解码
    else:
        print(recv_data, len(recv_data))
        print("对方已经挂机了")

    # 5 通信完成　挂机
    tcp_socket.close()
    # windows调试助手　发过来是GBK编码　收UTF-8编码的数据
if __name__ == '__main__':
    main()
```



### 4.3 服务器通信过程

```python
import socket


def main():
    # 1 大堂经理　　不负责具体业务　由业务员操作　－创建监听套接字
    listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2 确定地址　　－　绑定端口 参数是本机的IP和端口的元组　
    # 作为服务器来说一般都是绑定本机所有IP 填''
    listen_socket.bind(('', 8888))

    # 3 开始营业　　－　开始监听用户的连接请求 参数就是等待服务区的大小　经验值
    listen_socket.listen(128)

    while True:
        # 4 将用户请求转接到业务员　　－　开始接受用户的请求88888 ８８８８8888888
        # (<socket.socket服务器中 用以和客户端通信使用的套接字对象>, 客户端地址('172.16.51.151', 50224))
        server_client_socket, client_address = listen_socket.accept()
        print("接受到了来自%s的连接请求" % str(client_address))

        # 5 和用户进行通信
        recv_data = server_client_socket.recv(4096)
        print("接受到了数据 %s" % recv_data)

        server_client_socket.send(recv_data)

        # 6 关闭套接字
        server_client_socket.close()

if __name__ == '__main__':
    main()
```



### 4.4 地址重用选项

设置套接字选项   Address already in use

1 之前开的服务区没关

2 pycharm 后台进程还在跑

3 TCP规定问题

产生原因:  TCP规定 TCP 服务器重启<关闭 启动绑定之前同样的端口>需要在 30 秒到 2min时间 不能重新绑定这个端口

解决:         设置地址重用<让系统忽略这个时间> 立即重新绑定之前的端口

### 4.5 案例

协程版本多任务 TCP 服务器

```python
def 处理客户请求的函数():
    pass


def main():
    pass
	# 创建套接字  绑定监听
    # 接到用户请求时  在新协程中执行 处理客户请求的函数
```



```python
from gevent import monkey
monkey.patch_all()   #
import gevent
import socket
def client_request(server_client_socket):
    # 5 和用户进行通信
    while True:
        recv_data = server_client_socket.recv(4096)
        if recv_data:
            print("接受到了数据 %s" % recv_data.decode())
            server_client_socket.send(recv_data)
        else:
            print("用户已经下载")
            # 6 关闭套接字
            server_client_socket.close()
            break
def main():
    # 1 大堂经理　　不负责具体业务　由业务员操作　－创建监听套接字
    listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 设置地址重用选项set socket option  套接字选项  地址重用选项          开启选项
    listen_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)

    # 2 确定地址　　－　绑定端口 参数是本机的IP和端口的元组　
    # 作为服务器来说一般都是绑定本机所有IP 填''
    listen_socket.bind(('', 8888))
    # 3 开始营业　　－　开始监听用户的连接请求 参数就是等待服务区的大小　经验值
    listen_socket.listen(128)
    while True:
        # 4 将用户请求转接到业务员　　－　开始接受用户的请求88888 ８８８８8888888
        # (<socket.socket服务器中 用以和客户端通信使用的套接字对象>, 客户端地址('172.16.51.151', 50224))
        server_client_socket, client_address = listen_socket.accept()
        print("接受到了来自%s的连接请求" % str(client_address))
        # 当接受到用户的连接请求时  启动一个协程运行 client_request
        # 这里主线程除了启动协程 还有接受请求的任务 不需要join
        gevent.spawn(client_request, server_client_socket)


if __name__ == '__main__':
    main()

## 1. 写在最前

GIL问题

协程  用户概念

​	monkey.patch_all()   #  

​	协程对象 = gevent.spawn(入口,xxx,x,x)

​	协程对象.join()

基础概念

​	IP 地址作用、分类、查看IP ifconfig、测试网络连通情况  ping 

>   127.0.0.1 表示本机IP "我"  ping 127.0.0.1

​	端口作用

​	套接字   端到端 网络通信

TCP 特点

​	面向连接   建立连接、使用、关闭连接

​	可靠

​	基于字节流  无消息边界



客户端流程

```python
1 创建套接字
套接字对象 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
2 建立连接
套接字对象.connect(('服务器 IP', 端口))
3 发送数据
套接字对象.send(字节数据)
4 接收数据  正常就是字节数据 对方关闭了就是 b''
值 = 套接字对象.recv(最大字节数)
5 关闭
套接字对象.close()
```

服务器流程

```python
# 1 创建监听套接字
监听套接字对象 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 2 套接字设置地址重用选项
监听套接字对象.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# 3 绑定
监听套接字对象.bind(('', 服务器端口))

# 4 监听
监听套接字对象.listen(等待服务区大小)

# 5 接受一个用户的连接请求   重复执行
和客户端通信的套接字，客户端地址= 监听套接字对象.accept()

和客户端通信的套接字.recv()  .send()

和客户端通信的套接字.close()
```



地址重用



## 2. HTTP 协议-重点

>   开发、面试 过程中的  必备、重点知识 
>
>   HTTP 底层是基于TCP实现的<要想传输 HTTP网页资源数据 必须先建立连接>

### 2.1 简介

概念:HyperText Transfer Protocol 超文本传输协议

​	*超文本标记语言*(Hyper Text Markup Language)就是编写网页的技术

​	HTTP 作用: web浏览器 和 web服务器 之间  传输网页相关资源的协议

​	web 服务器: 提供网页资源的服务器程序

### 2.2 浏览器和服务器通信过程

1 地址栏输入网址(域名)  解析出域名

​	http://www.baidu.com

​	http://www.xhfjds.com:8080

2 找 DNS 服务器进行 DNS 解析返回 IP
3 使用IP+端口 连接 web 服务器
4 发送请求(HTTP 协议请求报文格式)
5 服务器解析请求 准备相应资源数据
6 服务器返回数据(HTTP 协议响应报文格式) 
7 浏览器接收、解析数据、展示数据
	5-7 只能完成一个资源的请求
	如果有多个资源 需要重复 567步直到完成
8 完成后 在合适的时机关闭连接



### 2.3 URL

作用： 定位互联网的资源   统一资源定位器

https://rpic.douyucdn.cn/live-cover/roomCover/2019/04/05/bc23723007dcac2e85539ff4821a3747_big.jpg?count=20

https://docs.python.org/3/search.html?q=asyncio&check_keywords=yes&area=default#threading

协议 https://

主机域名(IP):  rpic.douyucdn.cn

资源路径: /live-cover/roomCover/2019/04/05/bc23723007dcac2e85539ff4821a3747_big.jpg

​	以/开始

查询参数:?count=20

​	以?k=v&k1=v1&k2=v2

锚点   告诉浏览器这个网页 从哪儿开始显示

### 2.4 HTTP请求报文格式说明

```http
# 请求行 request line分为三步(请求方法 请求路径 协议版本)
GET / HTTP/1.1
# 请求方法:GET 一般用以浏览器从服务器获取资源
#    POST 一般用以浏览器向服务器提交资源
#    HEAD 网络服务的验证、测试
# 资源路径: https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&rsv_idx=1&tn=baidu&wd=haha&rsv_pq=eeaaede000019b65&rsv_t=b811dqJwygJu%2BAm9PB343MDpHU%2BOz4L%2BG34ovm8QsVMBA%2F4rCDI4zvD2yaI&rqlang=cn&rsv_enter=1&rsv_dl=tb&rsv_sug2=0&inputT=2471&rsv_sug4=2473 
	http://www.baidu.com         /
# GET /s?ie=utf-8&f=8&rsv_bp=1&rsv_idx=1&tn=baidu&wd=haha&rsv_pq=eeaaede000019b65&rsv_t=b811dqJwygJu%2BAm9PB343MDpHU%2BOz4L%2BG34ovm8QsVMBA%2F4rCDI4zvD2yaI&rqlang=cn&rsv_enter=1&rsv_dl=tb&rsv_sug2=0&inputT=2471&rsv_sug4=2473 HTTP/1.1
# 请求头格式:  键: 值
# 浏览器请求的服务器主机名(IP) + 非默认端口
Host: www.baidu.com
# 连接   长连接-保持存活keep-alive/短连接-立即关闭close
Connection: keep-alive
# 用户代理  浏览器身份信息
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36
# 浏览器可以接受数据类型
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3
# 浏览器可以接受数据的压缩格式
Accept-Encoding: gzip, deflate, br
# 浏览器可以接受数据的语言
Accept-Language: zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7
```



### 2.5 HTTP 请求报文格式总结

```http
请求行[请求方法 路径 协议版本\r\n]  一个
请求头[请求头: 值\r\n]  可以有任意个
空行[\r\n] 一个
请求体[浏览器需要提交给服务器的任意类型的数据 比如图片]一个
```

```http
GET / HTTP/1.1\r\n
Host: 127.0.0.1\r\n
Connection: keep-alive\r\n
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8\r\n
Upgrade-Insecure-Requests: 1\r\n
User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36\r\n
Accept-Encoding: gzip, deflate, sdch\r\n
Accept-Language: zh-CN,zh;q=0.8\r\n
\r\n
```

### 2.6 GET 和 POST 区别

>   面试必问

    ​			GET 		                                        POST

    ​	作用: 获取服务器资源   							      向服务器提交资源

    ​	格式: 一般不需要请求体       							 POST 一般都有请求体

    ​	应用场景: 大(获取资源)小(提交资源路径?k=v&)              提交的数据在请求体中

    ​	是否可以被缓存: 提交的数据被缓存在路径中                 数据不能被缓存

    ​	提交的数据类型:  键值对                                  任意类型



### 2.7 HTTP 响应报文

```http
# 状态行 status line 起始行 start line分为三部分
HTTP/1.1 200 OK
# 状态码 说明
# 1xx   继续   
# 2xx   成功   200 OK
# 3xx   重定向  HTTP/1.1 307 Internal Redirect
#              Location: https://www.baidu.com/
# 4xx   客户端错误 404 Not Found
# 5xx   服务器错误 500服务器错误 503 Service Unavailable
# 都是响应头
# 连接方式 长连接 close 短连接
Connection: Keep-Alive
# 内容-实体  压缩格式      实体(请求体、响应体)
Content-Encoding: gzip
Content-Length: 110  # 内容的长度 单位字节
Content-Language: zh-cn  # 内容采用的语言
# 内容数据类型
Content-Type: text/html content-type: image/png
# 日期  东8区
Date: Wed, 21 Aug 2019 03:05:15 GMT
# 过期日期
Expires: Wed, 21 Aug 2019 03:15:15 GMT
# 后台 web 服务器的名字  DIY  BaiduWebBServer
Server: BWS/1.1
```

### 2.8 HTTP响应报文格式总结

````http
状态行[版本 状态码 状态说明\r\n] 1 个
响应头[头: 值\r\n]  任意多个
空行[\r\n]
响应体[内容 服务器给浏览器返回网页资源数据 图片网页cssjs]
````

### 2.9 验证 HTTP 协议

```python
import socket
if __name__ == '__main__':
    # 1 模拟实现浏览器  创建套接字
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2 连接服务器
    tcp_socket.connect(('tlias3.boxuegu.com', 80))
    # 3 发送请求报文
    http_request_data = "GET / HTTP/1.1\r\nHost: tlias3.boxuegu.com\r\n\r\n"
    tcp_socket.send(http_request_data.encode())
    # 4 接收响应报文
    http_response_data = tcp_socket.recv(4096)
    print(http_response_data)
    # 5 关闭
    tcp_socket.close()
```

```
HTTP/1.1 302 Found\r\n
Date: Wed, 21 Aug 2019 03:48:58 GMT\r\n
Content-Type: text/html\r\n
Content-Length: 235\r\n
Connection: keep-alive\r\n
Location: https://tlias3.boxuegu.com/\r\n
Via: HTTP/1.1 SLB.33\r\n
\r\n
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\r\n<html>\r\n<head><title>302 Found</title></head>\r\n<body bgcolor="white">\r\n<h1>302 Found</h1>\r\n<p>The requested resource resides temporarily under a different URI.</p>\r\n</body>\r\n</html>\r\n
```

## 3. 自己动手实现 web 服务器

>   绑定著名端口 在 mac 和 linux 中都需要 root权限

web服务器 =  TCP 服务器 + HTTP 协议

>   HTTP状态码  https://www.runoob.com/http/http-status-codes.html

### 3.1 helloworld 版本

需求: 当接收到用户请求时 返回响应报文< helloworld作为响应体>

```python
import socket

#  1.0 当接收到用户请求时 返回响应报文< helloworld作为响应体>

def main():
    pass
    # 1 创建监听套接字 设置套接字重用选项  绑定监听
    listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    listen_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listen_socket.bind(('', 8888))
    listen_socket.listen(128)

    while True:
        # 2 接受用户的连接请求
        server_client_socket, client_address = listen_socket.accept()
        print("接受到了来自%s的连接请求" % str(client_address))

        # 3 接收用户的HTTP请求报文数据  解析处理 获取用户的目的 一定要接收
        http_request_data = server_client_socket.recv(4096)

        # 4 准备数据  以HTTP响应报文格式发送给浏览器
        http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\nhello world'
      server_client_socket.send(http_response_data.encode())

        # 5 关闭套接字
        server_client_socket.close()

if __name__ == '__main__':
    main()
```



### 3.2 返回固定资源

```python
import socket

#  1.0 当接收到用户请求时 返回响应报文< helloworld作为响应体>
#  2.0 当接收到用户请求时 返回响应报文<固定图片作为响应体>

def main():
    # 1 创建监听套接字 设置套接字重用选项  绑定监听
    listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    listen_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listen_socket.bind(('', 8888))
    listen_socket.listen(128)

    while True:
        # 2 接受用户的连接请求
        server_client_socket, client_address = listen_socket.accept()
        print("接受到了来自%s的连接请求" % str(client_address))

        # 3 接收用户的HTTP请求报文数据  解析处理 获取用户的目的 一定要接收
        http_request_data = server_client_socket.recv(4096)

        # 4 准备数据  以HTTP响应报文格式发送给浏览器
        # 响应体中可以放任意类型的数据　　发给浏览器
        # http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\nhello world'
        http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\n'
        # 从文件读取出资源数据
        with open("static/xjj.jpg", "rb") as file:
            response_body = file.read()  # 读出来就是字节数据  rb

        server_client_socket.send(http_response_data.encode() + response_body)

        # 5 关闭套接字
        server_client_socket.close()

if __name__ == '__main__':
    main()
```



### 3.3 返回指定资源

需求: 返回用户需要的对应资源

http://127.0.0.1:8888/index.html?k=v#xxx

http://127.0.0.1:8888/xjj.jpg



实现思路: 

​	1 从请求报文中获取到资源请求路径  /index.html 或者 /xjj.jpg

​	2 根据用户的资源读取出对应的资源数据

​	3 将资源数据按照响应报文组装 发送给客户端

实现:

```python
import socket

#  1.0 当接收到用户请求时 返回响应报文< helloworld作为响应体>
#  2.0 当接收到用户请求时 返回响应报文<固定图片作为响应体>
#  3.0 根据用户的具体资源路径请求 返回相应报文<指定资源作为响应体>

def main():
    # 1 创建监听套接字 设置套接字重用选项  绑定监听
    listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    listen_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listen_socket.bind(('', 80))
    listen_socket.listen(128)

    while True:
        # 2 接受用户的连接请求
        server_client_socket, client_address = listen_socket.accept()
        print("接受到了来自%s的连接请求" % str(client_address))

        # 3 接收用户的HTTP请求报文数据  解析处理 获取用户的目的 一定要接收
        # 3.1 接收请求报文
        http_request_data = server_client_socket.recv(4096)
        # 3.2 如果数据为空 意味着浏览器中途关闭了
        if not http_request_data:
            print("客户端已经关闭了")
            server_client_socket.close()
            return
        # 3.3 从请求报文中解析出请求行的  用户的资源路径
        path_info = http_request_data.decode().split("\r\n")[0].split(" ")[1]
        print("接收到了用户的请求 %s" %path_info)

        # 3.4 如果用户请求路径是　/  －－－－－－－－> 指定到主页
        if path_info == '/':
            path_info = '/grand.html'

        # 4 准备数据  以HTTP响应报文格式发送给浏览器
        # 响应体中可以放任意类型的数据　　发给浏览器
        try:
            # 可能会抛出异常的代码
            # 4.1 准备响应报文
            # http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\nhello world'
            http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\n'
            # 4.2 从用户指定的文件读取出对应的　资源数据
            # with open("static/xjj.jpg", "rb") as file:
            with open("static" + path_info, "rb") as file:
                response_body = file.read()  # 读出来就是字节数据  rb
        except Exception as e:
            # 如果有异常执行这里的代码
            print("%s" % str(e))
            http_response_data = 'HTTP/1.1 404 Not Found\r\nServer: PWS1.0\r\n\r\n'
            with open("static/404.html", "rb") as file:
                response_body = file.read()  # 读出来就是字节数据  rb
        else:
            # 如果没有发生异常就会执行这里
            pass
        finally:
            # 不管有没有发生异常都会执行这里
            # 4.3 将报文拼接完成发送给浏览器
            server_client_socket.send(http_response_data.encode() + response_body)

            # 5 关闭套接字
            server_client_socket.close()

if __name__ == '__main__':
    main()
```



### 3.4 多任务版本

>   这里用多线程实现    作用用协程实现

思路:

​	1 将处理用户请求的代码封装到一个函数

​	2 

实现:

```python
import socket

#  1.0 当接收到用户请求时 返回响应报文< helloworld作为响应体>
#  2.0 当接收到用户请求时 返回响应报文<固定图片作为响应体>
#  3.0 根据用户的具体资源路径请求 返回相应报文<指定资源作为响应体
#  4.0 多线程实现　　　　作业用协程实现  handler处理者　　处理逻辑

import threading
import time


# 处理客户端请求的函数
def client_request_handler(server_client_socket):
    # 3 接收用户的HTTP请求报文数据  解析处理 获取用户的目的 一定要接收
    time.sleep(5)
    # 3.1 接收请求报文
    http_request_data = server_client_socket.recv(4096)
    # 3.2 如果数据为空 意味着浏览器中途关闭了
    if not http_request_data:
        print("客户端已经关闭了")
        server_client_socket.close()
        return
    # 3.3 从请求报文中解析出请求行的  用户的资源路径
    path_info = http_request_data.decode().split("\r\n")[0].split(" ")[1]
    print("接收到了用户的请求 %s" % path_info)

    # 3.4 如果用户请求路径是　/  －－－－－－－－> 指定到主页
    if path_info == '/':
        path_info = '/grand.html'

    # 4 准备数据  以HTTP响应报文格式发送给浏览器
    # 响应体中可以放任意类型的数据　　发给浏览器
    try:
        # 可能会抛出异常的代码
        # 4.1 准备响应报文
        # http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\nhello world'
        http_response_data = 'HTTP/1.1 200 OK\r\nServer: PWS1.0\r\n\r\n'
        # 4.2 从用户指定的文件读取出对应的　资源数据
        # with open("static/xjj.jpg", "rb") as file:
        with open("static" + path_info, "rb") as file:
            response_body = file.read()  # 读出来就是字节数据  rb
    except Exception as e:
        # 如果有异常执行这里的代码
        print("%s" % str(e))
        http_response_data = 'HTTP/1.1 404 Not Found\r\nServer: PWS1.0\r\n\r\n'
        with open("static/404.html", "rb") as file:
            response_body = file.read()  # 读出来就是字节数据  rb
    else:
        # 如果没有发生异常就会执行这里
        pass
    finally:
        # 不管有没有发生异常都会执行这里
        # 4.3 将报文拼接完成发送给浏览器
        server_client_socket.send(http_response_data.encode() + response_body)

        # 5 关闭套接字
        server_client_socket.close()

def main():
    # 1 创建监听套接字 设置套接字重用选项  绑定监听
    listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    listen_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listen_socket.bind(('', 80))
    listen_socket.listen(128)

    while True:
        # 2 接受用户的连接请求
        server_client_socket, client_address = listen_socket.accept()
        print("接受到了来自%s的连接请求" % str(client_address))
        # 把所有跟用户请求处理相关的逻辑放到client_request_handler函数中 子线程运行
        # 一旦有新用户连接到服务器 我们就为这个用户创建一个线程处理逻辑
        thd = threading.Thread(target=client_request_handler, args=(server_client_socket,))
        thd.start()

if __name__ == '__main__':
    main()
```

